# D-AT-GRU
## Imports

In [1]:
%matplotlib inline
import itertools
import math
import time
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
from semeval2014.semeval_base import *

## Load Dataset

In [3]:
trainfile="semeval2014/restaurants-trial.xml"
testfile="semeval2014/Restaurants_Test_Data_PhaseA.xml"
corpus = Corpus(ET.parse(trainfile).getroot().findall('sentence'))
unseen = Corpus(ET.parse(testfile).getroot().findall('sentence'))
b1 = BaselineAspectExtractor(corpus)
predicted = b1.tag(unseen.corpus)
corpus.write_out('test.predicted-aspect.xml', predicted, short=False)

## Show 10 sentences and categories example in train

In [14]:
for index, sentence in zip(range(10), corpus.corpus):
    print(sentence.text)
    for categorie in sentence.aspect_categories:
        print(categorie.term, categorie.polarity)
    print("")

All the appetizers and salads were fabulous, the steak was mouth watering and the pasta was delicious!!!
food positive

And really large portions.
food positive

Go inside and you won't want to leave.
anecdotes/miscellaneous positive

Save yourself the time and trouble and skip this one!
anecdotes/miscellaneous negative

The sweet lassi was excellent as was the lamb chettinad and the garlic naan but the rasamalai was forgettable.
food conflict

Service was quick.
service positive

Oh, don't even let me start with how expensive the bills were!
price negative

Service is top notch.
service positive

The best thing I tasted were the lambc hops.
food positive

Overall I would recommend it and go back again.
anecdotes/miscellaneous positive



## Load Pre-trained Embedding Vectors from Glove (little dataset)

In [15]:
embeddings_dict = {}
with open("embeddings/glove.6B.300d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

NameError: name 'np' is not defined

## 